<a href="https://colab.research.google.com/github/anmaxwell/quotemaker/blob/master/Fine_Tuning_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

Copied from notebook by [Max Woolf](http://minimaxir.com).  Max's notebook is the basis of this notebook - I just removed a couple of the additional options he put in.

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use his notebook!



In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

I am using the 'smallest' model - `124M` as this is just to try out making quotes.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 261Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.06Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 389Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 29.7Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 587Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.43Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 2.81Mit/s]


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [4]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [5]:
file_name = "quotes.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [6]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:05<00:00,  5.95s/it]


dataset has 1222583 tokens
Training...
[10 | 28.21] loss=3.26 avg=3.26
[20 | 50.56] loss=3.25 avg=3.26
[30 | 73.34] loss=3.25 avg=3.25
[40 | 96.87] loss=3.25 avg=3.25
[50 | 121.29] loss=3.31 avg=3.26
[60 | 145.23] loss=3.25 avg=3.26
[70 | 168.95] loss=3.23 avg=3.26
[80 | 192.94] loss=3.27 avg=3.26
[90 | 217.04] loss=3.15 avg=3.25
[100 | 241.01] loss=3.14 avg=3.24
[110 | 264.94] loss=3.36 avg=3.25
[120 | 288.87] loss=3.19 avg=3.24
[130 | 312.83] loss=3.17 avg=3.24
[140 | 336.82] loss=3.23 avg=3.24
[150 | 360.79] loss=3.23 avg=3.23
[160 | 384.80] loss=3.21 avg=3.23
[170 | 408.79] loss=3.13 avg=3.23
[180 | 432.80] loss=3.00 avg=3.21
[190 | 456.83] loss=3.13 avg=3.21
[200 | 480.88] loss=3.06 avg=3.20
======== SAMPLE 1 ========
 income and education would be a great benefit to society.
There's also an element of fantasy. It's not about where you go and what you're capable of doing. But it's about who you are and why you come here, who you're with and your future.
The best thing I can offer 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [8]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [9]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [2]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [3]:
gpt2.generate(sess, run_name='run1')

I've always said that one way I can get out of a bad situation is to quit it. That can be an absolute death sentence. For me, that's the only way out of a bad situation.
Households are not going to pay more in gasoline or electricity. The money they currently have to use the utilities they own now costs them much more than they did before the Great Recession hit.
In America, I'm not the poster child for energy efficiency. But I'm the poster boy for energy conservation.
I don't think that we are on the verge of a climate catastrophe because we continue to expand our oil and gas industry despite a $4.7 billion deficit, huge budget deficits and a $1.2 billion budget deficit in the third quarter.
If you're a McDonald's customer, you know that there's nothing better or more fun than finding a fast food place to eat in your home. One of the things that McDonald's does is they take their customers and give them a reason to stay, to stay and go, and that's what we do.
If you're a McDonald's cu

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [7]:
gpt2.generate(sess,
              length=100,
              temperature=0.7,
              truncate='.',
              nsamples=15,
              batch_size=5
              )

I'm going to give you all a piece of my mind
What does that mean? What does that mean? You can't have a revolution unless you have a revolution
I think that the only way that we can really change is to change the culture
The light is blue and the darkness is gray
For my birthday, I got an old poster of 'The Art of Happiness' and I couldn't believe it
I am a very shy person
I saved money on clothes and car payments by buying a house and moving out
The idea of a nation that believes in free markets and creative innovation, with a focus on education and innovation, is the prudent course
I'm just a normal kid, but I can't wait to have my own apartment
The debate over abortion has reached a cross-partisan cross-country cross-country cross-country cross-country cross-country point of view
I am really happy living in a civilization where beauty is not a luxury
The full meaning of life is at stake when you decide to engage in the most radical changes in the pursuit of that ideal
The thing I re

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.